# Introduction to statistical features and features with scanpaths

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter("ignore")

## Getting simple dataset to work with. 

In [2]:
participant = 'Participant'
text = 'tekst'
x = 'norm_pos_x'
y = 'norm_pos_y'
t = 'start_timestamp'

In [3]:
data = pd.read_csv("../data/fixations/fixations_subset.csv")
data.head()

,Participant,id,duration,confidence,start_frame_index,start_timestamp,end_frame_index,dispersion,norm_pos_x,norm_pos_y,tekst,id_news,AOI
0,1,998,208.1115,0.999697,1806,317242.694809,1807,1.330883,0.242478,0.508895,1,1.0,C
1,1,999,209.2905,1.000000,1807,317242.913454,1808,1.650276,0.246931,0.433742,1,1.0,N
2,1,1000,235.8615,1.000000,1809,317243.128721,1810,1.249983,0.240074,0.408932,1,1.0,N
3,1,1001,231.0985,0.999868,1810,317243.376751,1812,1.097607,0.203017,0.505386,1,1.0,C
4,1,1002,225.3285,0.999293,1812,317243.633292,1813,1.206289,0.272986,0.503568,1,1.0,C


##### In order to extract features using the EyeFeatures methods, we only need the following columns: coordinates of fixations on the screen (that is x, y coordinates) and columns that identify the unique objects in the dataset. You can preprocess a dataset of raw gazes into the required format using a preprocessing module.

## Statistical Features

##### Computes statistical features regarding saccades, fixations, as well as microsaccades and regressions such as max length, mean acceleration, and other which are available in `stats` module of `eyefeatures.features`. 

##### **Note**: One can calculate statistics using any aggregation function supported by `pandas`.

In [4]:
import eyefeatures.features.stats as eye_stats

##### Here's an example of how saccades can be computed: desired features should be represented as a dictionary, with saccade properties as keys and lists of statistics as values.

##### In this example we prepare to calculate saccade length, speed and acceleration features, where

$$
\text{Length(Saccade}_i\text{)} = ||\text{Fixation}_{i+1} - \text{Fixation}_{i} ||_{2}\, \quad \text{Speed(Saccade}_i\text{)} = \frac{\text{Length(Saccade}_i\text{)}}{\text{Time}_{i+1} - \text{Time}_{i}}, \quad 
\text{Acceleration(Saccade}_i\text{)} = \frac{1}{2} \frac{\text{Speed(Saccade}_i\text{)} }{\text{Time}_{i+1} - \text{Time}_{i}}
$$
 
 
Reference: <i>Salvucci, D. D., & Goldberg, J. H. (Year). Identifying fixations and saccades in eye-tracking protocols. https://doi.org/10.1145/355017.355028 </i>

In [5]:
sac_feats_stats = {
    'length': ['min', 'max'],
    'speed': ['mean', 'kurtosis'],
    'acceleration': ['mean']
}

##### Finally, define a transformer and get the desired features.

In [6]:
sf = eye_stats.SaccadeFeatures(
    x=x,
    y=y,
    t=t,
    pk=[participant, text],
    features_stats=sac_feats_stats
)

##### Here, each row represents the features of one of 15 different `SUBJ` groups. 

In [7]:
sf.fit_transform(data)

,sac_length_min,sac_length_max,sac_speed_mean,sac_speed_kurtosis,sac_acceleration_mean
1_1,0.002657,0.433486,0.488956,5.498018,1.522857
1_2,0.003757,0.321837,0.451773,8.356764,1.507806
1_3,0.003663,0.365776,0.390753,7.639049,1.292694
1_4,0.000212,0.342315,0.333495,9.593384,0.999054
1_5,0.002705,0.375434,0.360223,9.717641,1.187418
...,...,...,...,...,...
2_33,0.001920,0.406550,0.264515,10.235914,0.945929
2_34,0.007928,0.354386,0.298396,9.732734,0.883503
2_35,0.002170,1.269198,0.623824,45.476927,2.527952
2_36,0.002196,1.124204,0.443600,10.414252,1.650884


## Scanpath Measures

##### This module offers classes and methods which calculate various measures of scanpaths. 

In [8]:
import eyefeatures.features.measures as eye_measures

##### Let's calculate some basic measures in order to demonstrate the usecase process.

##### The HurstExponent class estimates the Hurst exponent of a time series using Rescaled Range (R/S) analysis. The Hurst exponent is a measure of the long-term memory of time series data, indicating whether the data is trending, mean-reverting, or behaving in a random walk. Parameter `n_iters` regulates the number of iterations for the R/S analysis, while `fill_strategy` is the strategy to adjust data to the power of 2.

##### The algorithm is derived from the Rescaled Range (R/S) Analysis and works as follows:

* Time series is divided into segments (blocks) of equal size
* The mean is subtracted from each segment to center the data
* Compute the cumulative sum of the mean-adjusted data and determine the range (maximum - minimum) of the cumulative deviation
* Calculate the standard deviation of the original segment and the ratio of the range to the standard deviation
* The slope of he log of the block size and the log of the R/S ratio estimates the Hurst Exponent

$$
\log(R/S) = \text{HurstExponent} \cdot \log(n) + C, \, \text{where } R/S \text{ is the rescaled range}, \, n \text{ is the block size and } C \text{ is some constant}
$$

##### There are also some more complicated features.


##### One of them is RQA (Recurrence Quantification Analysis) for time-series or spatial data. 
The metrics calculated include Recurrence (REC), Determinism (DET), Laminarity (LAM), and Center of Recurrence Mass (CORM). These measures help to quantify the complexity and structure of the recurrence patterns within the data. In this example we use a default euclidean metric as `metric`. Parameters `rho` and `min_length` correspond for RQA matrix threshold radius and threshold length of its diagonal. In `measures` we specify the required features to calculate.

Recurrence matrix $R$ is defined as $R_{ij} = \mathbb{I}\left\{d(x_i, x_j) \leq \rho \right\}$:

- Reccurence Rate counts the total number of recurrence points above the main diagonal of $R$:
$$
\text{REC} = \frac{2}{n(n-1)} \sum_{i=1}^n \sum_{j=i+1}^n R_{ij}
$$
- Determinism measures the percentage of recurrence points forming diagonal lines of length at least $L_{min}$:
$$
\text{DET} = \frac{100 \cdot \sum_{l \geq L_{min}} l \cdot P(l)}{\sum_{i=1}^n \sum_{j=i+1}^n R_{ij}},
$$
$$
\text{ where } L_{min} - \text{ minimum line length}, \, P(l) - \text{probability of diagonal lines of length } l
$$
- Liminarity measures the percentage of recurrence points forming vertical or horizontal lines of length at least $L_{min}$:
$$
\text{LAM} = \frac{50 \left( \sum_{\text{HL}} \text{HL} + \sum_{\text{VL}} \text{VL}\right)}{\sum_{i=1}^n \sum_{j=i+1}^n R_{ij}},
$$
$$
\text{where HL and VL represents the sums of horizontal and vertical lines of length at least } L_{min}
$$
- Center of Recurrence Mass measures the weighted average of the distances between recurrence points, emphasizing the central tendency of recurrences in the matrix:
$$
\text{CORM} = \frac{100 \cdot \sum_{i=1}^{n-1} \sum_{j=i+1}^n (j-i) R_{ij}}{(n-1) \cdot \sum_{i=1}^n \sum_{j=i+1}^n R_{ij}}
$$

Reference: <i> Anderson, N. C., Bischof, W. F., Laidlaw, K. E. W., Foulsham, T., Kingstone, A., & Cristino, F. (2013). Recurrence quantification analysis of eye movements. Behavior Research Methods, 45(3), 842–856. https://doi.org/10.3758/s13428-012-0299-5 </i>

In [9]:
rqa_measures = eye_measures.RQAMeasures(
    metric=lambda p, q: np.linalg.norm(p - q),
    rho=0.10,
    min_length=1,
    measures=["rec", "corm"],
    x=x,
    y=y,
    pk=[participant, text],
    return_df=True
)

rqa_measures.fit_transform(data)

,rec_metric_<lambda>_length_1_rho_0.1,corm_<lambda>_length_1_rho_0.1
1_1,16.504988,25.985337
1_2,25.405961,27.918733
1_3,19.978675,21.611676
1_4,25.128498,26.572341
1_5,26.199669,24.729920
...,...,...
2_33,32.331793,31.245374
2_34,38.181818,30.972105
2_35,15.964397,31.556823
2_36,20.366972,26.304202


## Scanpath Distances

##### Let us describe the core idea of extrator-classes in this module. Each class calculates the "expected paths" for each path-group which are further used in distance functions. That is the resulting features for each group are simply the distances between two scanpaths: expected and given one.

Reference: <i> Fahimi, R., & Bruce, N. D. B. (2020). On metrics for measuring scanpath similarity. © The Psychonomic Society, Inc. https://doi.org/10.3758/s13428-020-01441-0   </i>

In [10]:
import eyefeatures.features.scanpath_dist as eye_dist

##### As for now, there are two ways to compute the expected path. 
- The first one simply aligns the paths by time and takes the pointwise mean at each timestamp. This method is used by passing `'mean'` to `expected_paths_method`
- The second algorithm seeks to find the so-called Fermat-Weber point (geometric median) of the series at each timestamp. This point basically minimizes the sum of distances from each observation. Use it by passing `'fwp'` to `expected_paths_method`.

##### See the example of calculating some basic distances. 
**Note:** primary key `path_pk` is set to be `'group'` so there is a separate expected path for each unique group. Primary key `pk` is also set to `['SUBJ', 'group']` which determines the way to distinguish between unique paths.

##### Explanation of distances used in this illustation:
- Euclidean distance is simply the sum of pairwise distances of two sequences at each timestamp:
$$
\text{EUC}(p, q) = \sum_{i=1}^n ||p_i - q_i||_2 \quad \text{ ($p$ and $q$ are alligned)}
$$
- EyeDist distance is calculated as follows:
$$
\text{EYE}(p, q) = \frac{1}{\max\{n, m\}} \left(\sum_{i=1}^n \min_{1 \leq j \leq m} ||p_i - q_j||_2^2 + \sum_{j=1}^m \min_{1 \leq i \leq n} ||q_j - p_i||_2^2\right)
$$
- Mannan distance is somewhat a more complex version of EyeDist since it considers the weighted distance:
$$
\text{MAN}(p, q) = \frac{1}{4 \cdot n \cdot m} \left(m \cdot \sum_{i=1}^n \min_{1 \leq j \leq m} ||p_i - q_j||_2^2 + n \cdot \sum_{j=1}^m \min_{1 \leq i \leq n} ||q_j - p_i||_2^2  \right)
$$

In [11]:
transformer = eye_dist.SimpleDistances(
    x=x,
    y=y,
    path_pk=[text],
    pk=[participant, text],
    methods=["euc", "eye", "man"],
    expected_paths_method="fwp",
    return_df=True
)

transformer.fit_transform(data)

100%|██████████| 74/74 [00:00<00:00, 771.95it/s]


,euc_dist_mean,eye_dist_mean,man_dist_mean
1_1,3.744766,0.004900,0.001225
1_2,4.193050,0.009499,0.002375
1_3,3.389498,0.014421,0.003605
1_4,2.031854,0.010721,0.002680
1_5,4.485001,0.009499,0.002375
...,...,...,...
2_33,20.057069,0.047082,0.011770
2_34,1.311675,0.081855,0.021456
2_35,15.454267,0.068109,0.019111
2_36,13.399683,0.041714,0.010429


##### Note that the expected paths are recalculated for each new distance class when the `fit` method is called, which takes up most of the runtime. To speed up the process, one can grep the expected paths from the previous class and reuse it with the new class. It is important that the primary keys for the new class match those of the previous class from which you obtained the expected paths.

In [12]:
expected_paths = transformer.expected_paths
expected_paths

{'1':         x_est     y_est
 0    0.259898  0.500364
 1    0.249939  0.471984
 2    0.245635  0.460574
 3    0.240511  0.528885
 4    0.266589  0.529640
 ..        ...       ...
 166  0.230134 -0.036350
 167  0.230783 -0.031528
 168  0.167029  0.132711
 169  0.167337  0.136862
 170  0.131066  0.171744
 
 [171 rows x 2 columns],
 '2':         x_est     y_est
 0    0.241289  0.505855
 1    0.283139  0.444494
 2    0.281076  0.418117
 3    0.295043  0.397100
 4    0.301977  0.393540
 ..        ...       ...
 135  0.184690  0.142985
 136  0.237087  0.021538
 137  0.255213  0.007728
 138  0.220779  0.057421
 139  0.183501  0.189892
 
 [140 rows x 2 columns],
 '3':         x_est     y_est
 0    0.218040  0.550639
 1    0.194212  0.518820
 2    0.222246  0.488000
 3    0.216926  0.531388
 4    0.248582  0.521505
 ..        ...       ...
 118  0.264671  0.005846
 119  0.253570 -0.050149
 120  0.253778 -0.039273
 121  0.220826  0.009683
 122  0.215113  0.033564
 
 [123 rows x 2 columns],
 '4'

##### The same logic can be applied to the filling path (`fill_path`), which is used when no expected path is found for a particular group whenever `transform` is called. It is calculated as the mean of all known expected paths (basically the expected path over the expected paths with `expected_paths_method` set to `'mean'`).

In [13]:
fill_path = transformer.fill_path
fill_path

,x_est,y_est
0,0.283867,0.611614
1,0.265157,0.587209
2,0.265402,0.577469
3,0.276539,0.581066
4,0.296046,0.577971
...,...,...
268,0.006425,0.007625
269,0.007180,0.007700
270,0.007204,0.007663
271,0.007066,0.004577


##### There are also methods in `scanpath_complex` module that return features not in the form of numbers that can be used for inference. This is usually some kind of structure (for instance, a matrix) that can be used for analyzing data or further feature extracting.

In [14]:
import eyefeatures.features.scanpath_complex as eye_complex

##### Let's see one of the possible usecases. 

##### We get the list of scanpaths of form `(x_coord, y_coord)` in order to calculate the pairwise distance matrix. One can use custom metric or those already implemented in `scanpath_dist`.

In [15]:
list_of_scanpaths = [scanpath[[x, y]].reset_index(drop=True) for _, scanpath in data.groupby(participant)]
len(list_of_scanpaths)

2

In [16]:
euc_matrix = eye_complex.get_dist_matrix(list_of_scanpaths, dist_metric=eye_dist.calc_euc_dist)
euc_matrix

100%|██████████| 2/2 [00:00<00:00, 4691.62it/s]


q,0,1
p,,
0,0.000000,16875.995776
1,16875.995776,0.000000


In [17]:
eye_matrix = eye_complex.get_dist_matrix(list_of_scanpaths, dist_metric=eye_dist.calc_eye_dist)
eye_matrix

100%|██████████| 2/2 [00:00<00:00,  9.28it/s]


q,0,1
p,,
0,0.000000,3.434076
1,3.434076,0.000000


##### Now we can calculate the compromise matrix using these pairwise distances matrices. 

##### The compromise matrix serves as a summary that captures the most common structure or pattern shared across all the individual distance matrices. It is used in applications where you need to summarize information from multiple sources (in our case these are different metrics), providing a single matrix that best represents the consensus of all input matrices.

##### Some elaboration on how compromise matrix is built:

- Given a weight vector $\mathbf{w} = [w_1, w_2, \dots, w_n]$ where $w_i \geq 0$ and $\sum_{i=1}^{n} w_i = 1$, the **centering matrix** $\Theta$ which centers the data is defined as:

$$
\Theta = I_n - \mathbf{1}_n \mathbf{w}^T
$$
$$
\text{where } I_n \text{ is the identity matrix of size } n \times n,\, \mathbf{1}_n \text{ is a column vector of ones of size } n \times 1
$$

- For a given distance matrix $D$ and weight vector $\mathbf{w}$, the **cross-product matrix** $S$ is calculated as:

$$
S = -\frac{1}{2} \Theta D \Theta^T
$$
$$
\text{where } D \text{ is the distance matrix}, \, \Theta \text{ is the centering matrix from previous step}
$$

- The **RV coefficient** measures the similarity between two cross-product matrices $S_1$ and $S_2$. It is defined as:

$$
\text{RV}(S_1, S_2) = \frac{\text{Tr}(S_1 S_2^T)}{\sqrt{\text{Tr}(S_1 S_1^T) \cdot \text{Tr}(S_2 S_2^T)}}
$$
$$
\text{where } \text{Tr}(\cdot) \text{ is the trace (sum of diagonal elements) of a matrix}
$$

- Finally, to obtain the compromise matrix, we first compute the **similarity matrix** using the RV coefficients between all pairs of cross-product matrices. Then, we perform **eigen-decomposition** on this similarity matrix to find the principal eigenvector $\mathbf{w}\,$:

$$
S_{\text{compromise}} = \sum_{i=1}^{k} w_i S_i
$$
$$
\text{ where} w_i \text{ is the weight from the principal eigenvector}, \, S_i \text{ is the cross-product matrix for the $i$-th distance matrix}
$$

In [18]:
eye_complex.get_compromise_matrix([euc_matrix, eye_matrix])

,0,1
0,2983.889828,-2983.889828
1,-2983.889828,2983.889828


##### One can also calculate a similarity matrix of the scanpaths. See the example of using it with a custom metric. 

In [19]:
def sim(p, q) -> float:
    return 1 / eye_dist.calc_euc_dist(p, q)

sim_matrix = eye_complex.get_sim_matrix(list_of_scanpaths, sim_metric=sim)
pd.DataFrame(sim_matrix)

,0,1
0,1.00000,0.50003
1,0.50003,1.00000


##### There are 4 methods implemented in `scanpath_complex` for similarity matrix reordering. For example, we can use `dimensionality_reduction_order` for the matrix calculated above. This function applies a dimensionality reduction technique, such as Multi-Dimensional Scaling (MDS), to the input similarity matrix. The goal is to project the items into a lower-dimensional space (typically 1D) where the order of items reflects their dissimilarities as closely as possible. The indices of items are then reordered according to their positions in this lower-dimensional space, resulting in an ordering that preserves the structure of the original similarities.

In [20]:
mds_reordered_matrix = eye_complex.dimensionality_reduction_order(sim_matrix)
pd.DataFrame(mds_reordered_matrix)

,0,1
0,1.00000,0.50003
1,0.50003,1.00000


## Extractor Class

##### Finally, we can combine several extractor classes into one `Extractor` class to calculate all the features at once.

In [21]:
from eyefeatures.features.extractor import Extractor

extractor = Extractor(
    features=[
        eye_dist.SimpleDistances(
            methods=["euc", "eye", "man"],
            expected_paths_method="fwp",
        ),
        eye_measures.GriddedDistributionEntropy(),
        eye_stats.SaccadeFeatures(
            features_stats=sac_feats_stats
        )
    ],
    x=x,
    y=y,
    t=t,
    duration='duration',
    dispersion='dispersion',
    path_pk=[text],
    pk=[participant, text],
    return_df=True
)

extractor.fit_transform(data)

100%|██████████| 74/74 [00:00<00:00, 3777.73it/s]

100%|██████████| 74/74 [00:00<00:00, 699.53it/s]

100%|██████████| 3/3 [00:00<00:00,  9.71it/s]


,euc_dist_mean,eye_dist_mean,man_dist_mean,gridded_entropy_grid_size_10,sac_length_min,sac_length_max,sac_speed_mean,sac_speed_kurtosis,sac_acceleration_mean
1_1,3.744766,0.004900,0.001225,3.983565,0.002657,0.433486,3.354343,5.209396,100.310810
1_2,4.193050,0.009499,0.002375,3.776755,0.003757,0.321837,2.790059,1.062294,82.774871
1_3,3.389498,0.014421,0.003605,3.696548,0.003663,0.365776,2.859377,3.861619,86.208709
1_4,2.031854,0.010721,0.002680,3.703330,0.000212,0.342315,2.753961,0.792342,96.485501
1_5,4.485001,0.009499,0.002375,3.751222,0.002705,0.375434,2.631501,2.786345,83.221221
...,...,...,...,...,...,...,...,...,...
2_33,20.057069,0.047082,0.011770,3.805528,0.001920,0.406550,2.544267,-0.174805,125.494989
2_34,1.311675,0.081855,0.021456,3.514732,0.007928,0.354386,2.776038,-0.412796,121.416459
2_35,15.454267,0.068109,0.019111,2.933468,0.002170,1.269198,3.458571,20.617815,141.254593
2_36,38.845117,0.263310,0.066319,3.045834,0.002196,1.124204,3.228725,23.727673,136.553104


##### Extractor class can be easily integrated into the `sklearn Pipeline` as it fully follows the sklearn API.

##### In this example, the pipeline calls the extractor to calculate the desired features first and then passes them to the model. Note that the extractor can save additional features from the input `DataFrame` before passing them to the model, if needed.

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('extractor', extractor),
    ('classifier', LogisticRegression())
])